# Cypher queries

## Testing import from blocks and transaction topics by sample queries

In [ ]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"))

## Example Data

In [ ]:
# block node
b1 = {
        "block_hash": "00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2",
        "block_height": "681545",
        "block_timestamp" : "2021-05-02 16:50:00",
        "p_block_hash": "00"
    }

b2 = {
        "block_hash": "00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395986",
        "block_height": "6815765",
        "block_timestamp" : "2021-05-02 17:00:00",
        "p_block_hash": "00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2"
    }

b3 = {
        "block_hash": "00000000000000000008bb27dd339ad1238912s73981273981d7298127491824",
        "block_height": "6815766",
        "block_timestamp" : "2021-05-02 17:00:00",
        "p_block_hash": "00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395986"
    }

# transaction node 
t0 = {
    "txid": "0",
    "block_date": "2021-05-02",
    "inDegree": 1,
    "outDegree": 2, 
    "inSum": 10,
    "outSum": 10
}

t1 = {
    "txid": "cebc8ac65d27d9ae05aad5249e5b991e01d50b256118ea1722373a212814b81e",
    "block_date": "2021-05-02",
    "inDegree": 1,
    "outDegree": 4, 
    "inSum": 20.0,
    "outSum": 13.07
}

# address node 
a1 = {
    "address": "1EepjXgvWUoRyNvuLSAxjiqZ1QqKGDANLW",
    "inDegree": 1, 
    "outDegree": 1
}

# FOREACH Test
f0 = {
    "txid" : "0",
    "addresses": [
        {
            "addr": "id_addr0",
            "output_nr": 0
        },
        {
            "addr": "id_addr1",
            "output_nr": 1
        },
        
    ], 
    "block_hash":'00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2'
    
}

f1 = {
    "txid" : "1",
    "addresses": [
        {
            "addr": "id_addr1_0",
            "output_nr": 0
        },
        {
            "addr": "id_addr1_1",
            "output_nr": 1
        },
        
    ],
    "input_txid":"0",
    "input_vout":1,
    "block_hash":'00000000000000000008bb27dd339ad1238912s73981273981d7298127491824'
    
}

## Transactions


In [ ]:
def create_transaction(tx, txid, block_date, inDegree, outDegree, inSum, outSum):
    tx.run('MERGE (t:Transaction{txid: $txid, block_date: $block_date, inDegree: $inDegree, outDegree: $outDegree, inSum: $inSum, outSum: $outSum})',
           txid=txid, block_date=block_date, inDegree=inDegree, outDegree=outDegree, inSum=inSum, outSum=outSum)

In [ ]:
t = t1
with driver.session() as session:
    session.write_transaction(create_transaction, 
                              t['txid'],
                              t['block_date'],
                              t['inDegree'],
                              t['outDegree'],
                              t['inSum'],
                              t['outSum'])

#### FOREACH Test

In [ ]:
def test_foreach(tx, txid, addresses):
    tx.run('MERGE (f:Transaction{txid: $txid}) '
           'FOREACH (address in $addresses | '
           ' MERGE (a:Address{txid: address.addr}) '
           ' MERGE (f)-[r:RECEIVES{output_nr: address.output_nr}]->(a))',
           txid=txid, addresses=addresses)

In [ ]:
query = '''
    MATCH (input_t:Transaction)-[input_r:RECEIVES]->(input_a:Address) 
        WHERE 
            input_t.txid ="0" AND 
            input_r.output_nr = 0 
            
    
    MERGE (t:Transaction{txid: $txid}) 
    
    MERGE (input_a)-[:Sends]->(t)
    MERGE (b:Block{hash: $blockhash})
    MERGE (t)-[:BELONGS_TO]->(b)
    
    FOREACH (address in $addresses | 
        MERGE (a:Address{txid: address.addr}) 
        MERGE (t)-[r:RECEIVES{output_nr: address.output_nr}]->(a))
    '''

def test_foreach(tx, txid, addresses, blockhash):
    tx.run(query, txid=txid, addresses=addresses, blockhash=blockhash)

In [ ]:
f = f1
with driver.session() as session:
    session.write_transaction(test_foreach, f["txid"], f["addresses"], f["block_hash"])

## Blocks

#### Creating initial block for backreferencing

In [ ]:
def create_initial_block(tx, initial_hash):
    tx.run('CREATE (b:Block{hash: $initial_hash})', initial_hash=initial_hash)

with driver.session() as session:
    session.write_transaction(create_initial_block, b1['p_block_hash'])

#### Function to create additional blocks

In [ ]:
def create_block(tx, block_hash, block_height, block_timestamp):
    tx.run('MERGE (b:Block{hash: $block_hash, height: $block_height, timestamp: $block_timestamp})',
           block_hash=block_hash, block_height=block_height, block_timestamp=block_timestamp)

In [ ]:
with driver.session() as session:
    session.write_transaction(create_block, b2['block_hash'], b2['block_height'], b2['block_timestamp'])

#### Block Creation and Back Referencing

The function creates the new block, matches the preceding block and creates the relationship between both.
<br>
Issue: breaks if preciding block not creates (not found(?))

In [ ]:
def create_block(tx, block_hash, block_height, block_timestamp, p_block_hash):
    tx.run('MATCH (p:Block) WHERE p.hash = $p_block_hash ' 
           'MERGE (b:Block{hash: $block_hash, height: $block_height, timestamp: $block_timestamp}) '
           'MERGE (p)-[r:Precedes]->(b)',
           block_hash=block_hash, block_height=block_height, block_timestamp=block_timestamp, p_block_hash=p_block_hash)

#### Alternative that recognizes whether the block was already created by the transaction

In [ ]:
query = '''
MATCH (p:Block) WHERE p.hash = $p_block_hash 
MERGE (b:Block {hash: $block_hash}) 
    SET b = {hash: $block_hash, height: $block_height, timestamp: $block_timestamp }
MERGE (p)-[r:Precedes]->(b)
'''

def create_block(tx, block_hash, block_height, block_timestamp, p_block_hash):
    tx.run(query, block_hash=block_hash, block_height=block_height, block_timestamp=block_timestamp, p_block_hash=p_block_hash)

In [ ]:
block = b3
with driver.session() as session:
    session.write_transaction(create_block, 
                              block['block_hash'], 
                              block['block_height'], 
                              block['block_timestamp'],
                              block['p_block_hash'])
    

#### Final Blocks query

In [ ]:
"""
MERGE (p:Block {hash: event.previousblockhash}) 
MERGE (b:Block {hash: event.block_hash}) 
SET b = {hash: event.block_hash, height: event.block_height, blockDate: event.block_date, mediantime: event.block_timestamp } 
MERGE (p)-[r:Precedes]->(b)

"""